In [1]:
from tqdm import tqdm
# from __future__ import print_function
# from __future__ import division
import torch
import pathlib2
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from src import constant
# from robustbench import load_model
from src.utils import get_model
import time
import glob
import os
import copy
from src.bootstrap import Cifar10Bootstrapper
from src.job import Cifar10Job
from src.utils import get_transformation_threshold, transform_image, transform_image_dir
import cv2
from src.dataset import GeneralDataset
from src.utils import start_matlab
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.9.0
Torchvision Version:  0.10.0


In [2]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "Kireev2021Effectiveness_Gauss50percent"
num_classes = 10
train_dataset_path = "./transformed_data/train"
val_dataset_path = "./transformed_data/val"
batch_size = 8
num_epochs = 10
feature_extract = False

In [3]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=25):
    model.eval()
    avg_loss, avg_acc = eval_model(dataloaders['val'], model, criterion, device)
    print(f"initial performance: avg_loss: {avg_loss}, avg_acc: {avg_acc}")
    since = time.time()
    
    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [4]:
def eval_model(dataloader, model, criterion, device):
    model.eval()
    running_loss = 0
    running_corrects = 0
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    avg_loss = running_loss / len(dataloader.dataset)
    avg_acc = running_corrects.double() / len(dataloader.dataset)
    return avg_loss, avg_acc

In [5]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

image_datasets = {
    'train': GeneralDataset(pathlib2.Path("./transformed_data/train")),
    'val': GeneralDataset(pathlib2.Path("./transformed_data/val"))
}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = get_model(model_name)
set_parameter_requires_grad(model_ft, feature_extract)
model_ft = model_ft.to(device)

In [7]:
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.0.conv2.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv1.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer1.1.conv2.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv1.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.conv2.weight
	 layer2.0.shortcut.0.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer2.1.conv2.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv1.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.conv2.weight
	 layer3.0.shortcut.0.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn2.weight
	 layer3.1.bn2.bias
	 layer3.1.conv2.weight
	 layer4.0.bn1.weight
	 layer4.0.bn1.bias
	 layer4.0.conv1.weight
	 layer4.0.bn2.weight
	 layer4.0.bn2.bias
	 layer4.0

In [8]:
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, device, num_epochs=num_epochs)

TypeError: eval_model() missing 1 required positional argument: 'device'

# Original Model + Transformed Dataset (val)

In [ ]:
original_model_ft = get_model(model_name)
original_model_ft = original_model_ft.to(device)
original_model_ft.eval()
# evaluate
avg_loss, avg_acc = eval_model(dataloaders_dict['val'], original_model_ft, criterion)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")

100%|██████████| 1250/1250 [00:05<00:00, 237.37it/s]

avg_loss: 0.6363577985350043, avg_acc: 0.8244


# Original Model on Original Dataset

In [ ]:
original_dataset = GeneralDataset(pathlib2.Path("./data/cifar10_pytorch/val"))
original_dataloader = torch.utils.data.DataLoader(original_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
avg_loss, avg_acc = eval_model(original_dataloader, original_model_ft, criterion)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")

100%|██████████| 1250/1250 [00:05<00:00, 226.15it/s]

avg_loss: 0.15374231626961846, avg_acc: 0.9508000000000001


# New Model on Original Dataset

In [ ]:
avg_loss, avg_acc = eval_model(original_dataloader, model_ft, criterion)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")

100%|██████████| 1250/1250 [00:05<00:00, 223.46it/s]

avg_loss: 0.4474117938951124, avg_acc: 0.8717


# Transformed Model on Cifar10C

In [ ]:
cifar10c_dataset = GeneralDataset(pathlib2.Path("./data/cifar-10-c-images/contrast"))
cifar10c_dataloader = torch.utils.data.DataLoader(cifar10c_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
avg_loss, avg_acc = eval_model(cifar10c_dataloader, model_ft, criterion)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")

100%|██████████| 6250/6250 [00:27<00:00, 228.66it/s]

avg_loss: 0.3023969431984052, avg_acc: 0.9136400000000001


# Original Model on Cifar10C

In [ ]:
avg_loss, avg_acc = eval_model(cifar10c_dataloader, original_model_ft, criterion)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")

100%|██████████| 6250/6250 [00:26<00:00, 238.41it/s]

avg_loss: 0.5895170183174032, avg_acc: 0.83194


# Result
| model    | dataset     | accuracy |
| -------- | ----------- | -------- |
| original | original    | 0.9508   |
| original | transformed | 0.8244   |
| original | cifar10c    | 0.83194  |
| fine     | original    | 0.8717   |
| fine     | transformed | 0.913900 |
| fine     | cifar10c    | 0.91364  |

